<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/livia-branch/AmazonDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [24.5 kB]
Hit:12 http://ppa.launchpad.ne

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Selected furniture data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_furniture = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_furniture.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [4]:
print((df_furniture.count(), len(df_furniture.columns)))

(792113, 15)


In [5]:
# Drop the non-beneficial columns
columns_to_drop = ["review_id","product_title","helpful_votes","review_headline","review_body","marketplace"]

df_furniture = df_furniture.drop(*columns_to_drop)
df_furniture.show()

+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|   24509695|B004HB5E0E|     488241329|       Furniture|          4|          0|   N|                Y| 2015-08-31|
|   34731776|B0042TNMMS|     205864445|       Furniture|          5|          0|   N|                Y| 2015-08-31|
|    1272331|B0030MPBZ4|     124663823|       Furniture|          5|          1|   N|                Y| 2015-08-31|
|   45284262|B005G02ESA|     382367578|       Furniture|          3|          0|   N|                Y| 2015-08-31|
|   30003523|B005JS8AUA|     309497463|       Furniture|          3|          0|   N|                N| 2015-08-31|
|   18311821|B00AVUQQGQ|     574537906|       Furniture|          5|    

In [6]:
# Only considering verified purchases
df_furniture = df_furniture.filter(df_furniture.verified_purchase == "Y")
print((df_furniture.count(), len(df_furniture.columns)))

(718192, 9)


In [7]:
# Selected home data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_home = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_home.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [8]:
print((df_home.count(), len(df_home.columns)))

(7013672, 15)


In [9]:
# Drop the non-beneficial columns
df_home = df_home.drop(*columns_to_drop)
df_home.show()

+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|   33670092|B00EE62UAE|     583436067|            Home|          1|          0|   N|                Y| 2015-08-31|
|   13726692|B001APXO5C|     465035091|            Home|          5|          9|   N|                Y| 2015-08-31|
|   50131396|B002HFDLCK|     136507891|            Home|          5|          0|   N|                Y| 2015-08-31|
|   16046884|B00PL9EFPQ|     631238459|            Home|          2|          0|   N|                Y| 2015-08-31|
|   11417282|B00B5TPVQS|     190431573|            Home|          4|          0|   N|                N| 2015-08-31|
|   48013430|B00NQDGHDC|     124050883|            Home|          5|    

In [10]:
# Only considering verified purchases
df_home = df_home.filter(df_home.verified_purchase == "Y")
print((df_home.count(), len(df_home.columns)))

(6263101, 9)


In [11]:
print((df_home.count(), len(df_home.columns)))

(6263101, 9)


In [53]:
# Merge both databases
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

df_merge = unionAll(df_furniture, df_home)
df_merge.show()


+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|   24509695|B004HB5E0E|     488241329|       Furniture|          4|          0|   N|                Y| 2015-08-31|
|   34731776|B0042TNMMS|     205864445|       Furniture|          5|          0|   N|                Y| 2015-08-31|
|    1272331|B0030MPBZ4|     124663823|       Furniture|          5|          1|   N|                Y| 2015-08-31|
|   45284262|B005G02ESA|     382367578|       Furniture|          3|          0|   N|                Y| 2015-08-31|
|   18311821|B00AVUQQGQ|     574537906|       Furniture|          5|          0|   N|                Y| 2015-08-31|
|   42943632|B00CFY20GQ|     407473883|       Furniture|          5|    

In [54]:
print((df_merge.count(), len(df_merge.columns)))

(6981293, 9)


In [55]:
# Get datatypes
df_merge.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_date: string (nullable = true)



In [56]:
from pyspark.sql.types import *
df_merge = df_merge.withColumn("review_date", df_merge["review_date"].cast(DateType()))

df_merge.printSchema()
df_merge.show()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_date: date (nullable = true)

+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|   24509695|B004HB5E0E|     488241329|       Furniture|          4|          0|   N|                Y| 2015-08-31|
|   34731776|B0042TNMMS|     205864445|       Furniture|          5|          0|   N|                Y| 2015-08-31|
|    1272331|B0030M

In [57]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
 
df_merge = df_merge.withColumn("review_year",year(df_merge.review_date))
df_merge.show()

+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+-----------+
|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|review_year|
+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+-----------+
|   24509695|B004HB5E0E|     488241329|       Furniture|          4|          0|   N|                Y| 2015-08-31|       2015|
|   34731776|B0042TNMMS|     205864445|       Furniture|          5|          0|   N|                Y| 2015-08-31|       2015|
|    1272331|B0030MPBZ4|     124663823|       Furniture|          5|          1|   N|                Y| 2015-08-31|       2015|
|   45284262|B005G02ESA|     382367578|       Furniture|          3|          0|   N|                Y| 2015-08-31|       2015|
|   18311821|B00AVUQQGQ|     574537906|       Furniture|          5|          0|   N|                Y| 

In [58]:
# Create a table to count data rows per year
row_year_table = df_merge.groupby("review_year").agg({"review_year":"count"}).withColumnRenamed("count(review_year)", "year_count")
row_year_table.sort(row_year_table.review_year.desc()).show(truncate=False)

+-----------+----------+
|review_year|year_count|
+-----------+----------+
|2015       |2534593   |
|2014       |2185166   |
|2013       |1345202   |
|2012       |468626    |
|2011       |234799    |
|2010       |118673    |
|2009       |44315     |
|2008       |22315     |
|2007       |17074     |
|2006       |5484      |
|2005       |2410      |
|2004       |1037      |
|2003       |795       |
|2002       |301       |
|2001       |102       |
|2000       |15        |
|null       |0         |
+-----------+----------+



In [59]:
# Only considering 2015 data
df_merge = df_merge.filter(df_merge.review_year == "2015")
print((df_merge.count(), len(df_merge.columns)))

(2534593, 10)


In [60]:
df_merge.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- review_year: integer (nullable = true)



# For additional data on other products segments:
https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt